## READ ME
- ランダム行列の生成は$A = \begin{pmatrix}
U(-h,h) & U(-h,h) \\
U(-h,h) & U(-h,h)
\end{pmatrix}$で$A$を生成し，$X = A {}^t\!A$として正定値行列を生成

## ライブラリのimport

In [52]:
# !pip install mip 
from mip import Model, maximize, minimize, xsum
import numpy as np
from numpy import pi
from numpy.random import rand, randn
from numpy.linalg import solve, det
from scipy.linalg import sqrtm
import math


## 定数の設定

In [53]:
p = 2 # 行列の次元
N = 500 # データ数
lam = 0.1 # 正則化パラメータ
delta = 10.0
epsilon = 1.0
h = 4.0 #U(-h,h)

## Wasserstein距離を計算する関数

In [54]:
def wasserstein(x, y):
    z = sqrtm(sqrtm(x) @ y @ sqrtm(x))
    res = np.trace(x) + np.trace(y) - 2 * np.trace(z)
    return np.sqrt(res)

## Entropy正則化されたWasserstein距離を計算する関数

In [55]:
def reg_wasserstein(x, y):
    M_lam = sqrtm(lam * lam * np.eye(p) + sqrtm(x) @ y @ sqrtm(x))
    res = np.trace(x) + np.trace(y) - 2.0 * np.trace(M_lam) - 2.0 * lam * math.log(det(M_lam - lam * np.eye(p)),math.e) - 2.0 * lam * p * math.log(2.0 * math.pi * lam, math.e) - 4.0 * lam * p * math.log(2.0 * math.pi, math.e) - 2.0 * lam * p + 2.0 * lam * math.log(det(x @ y),math.e) + 4.0 * lam * p * (math.log(2.0 * math.pi, math.e) + 1.0)
    return np.sqrt(res)

## 乱数の生成と距離グラフの生成

### wasserstein

In [ ]:
P = []
nbh_w = [[] for _ in range(N)]

i = 0
np.random.seed(123)
while i < N:
    a = np.array([[rand(1) * 2.0 * h - h, rand(1) * 2.0 * h - h],[rand(1) * 2.0 * h - h, rand(1) * 2.0 * h * h]])
    a = a.reshape((2,2)) 
    v = a @ a.T
    # B(I,delta)内になければskip
    if det(v) < 0.0001:
        continue 
    if wasserstein(np.eye(p),v) > delta :
        continue
    
    for j in range(0,i):
        w_dist = wasserstein(P[j],v)
        if w_dist < epsilon:
            nbh_w[i].append(j)
            nbh_w[j].append(i)
            
    nbh_w[i].append(i)
            
    i += 1
    P.append(v)

print(nbh_w)
print(len(P),len(nbh_w))
    

[[0, 19, 27, 76, 108, 155, 206, 259, 337, 433, 440, 450, 488], [1, 2, 356, 368, 408, 413, 490], [1, 2, 79, 176, 356, 368, 408, 490], [3, 23, 86, 129, 205, 268, 320, 379, 392, 432], [4, 74, 97, 255, 389, 422, 470], [5, 43, 178, 276, 309, 328, 377, 425], [6, 106, 173, 187, 311, 460, 465], [7, 62, 217, 249, 257, 367, 395, 419, 491], [8, 89, 130, 153, 157, 160, 165, 188, 191, 218, 244, 265, 348, 380, 451, 457], [9, 38, 50, 64, 121, 204, 235, 324, 384, 409, 441, 462], [10, 11, 84, 147, 153, 157, 188, 265, 298, 365, 454, 486], [10, 11, 153, 157, 215, 265, 365, 457], [12, 22, 142, 266, 481, 497], [13, 99, 125, 133, 137, 245, 458], [14, 27, 73, 76, 111, 126, 229, 259, 264, 299, 349, 351, 431, 433, 461, 463, 488], [15, 115, 152, 221, 310, 314, 387, 406, 452], [16, 70, 71, 134, 149, 152, 159, 163, 184, 262, 310, 406, 445], [17, 144, 172, 183, 200, 285, 316, 449, 456], [18, 55, 82, 93, 104, 224, 302], [0, 19, 55, 206, 337], [20, 201, 212, 270, 446], [21, 78, 116, 165, 386, 477], [12, 22, 142, 266

### regularized wasserstein

In [59]:
P = []
nbh_rw = [[] for _ in range(N)]


i = 0
np.random.seed(123)
while i < N:
    a = np.array([[rand(1) * 2.0 * h, rand(1) * 2.0 * h - h],[rand(1) * 2.0 * h - h, rand(1) * 2.0 *h]])
    a = a.reshape((2,2)) 
    v = a @ a.T
    # B(I,delta)内になければskip
    if reg_wasserstein(np.eye(p),v) > delta:
        continue
    
    for j in range(0,i):
        rw_dist = reg_wasserstein(P[j],v)
        if rw_dist < epsilon:
            nbh_rw[i].append(j)
            nbh_rw[j].append(i)
            
    nbh_rw[i].append(i)
            
    i += 1
    P.append(v)

print(nbh_rw)
print(len(P),len(nbh_rw))
    

[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24, 271], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40, 135], [41], [42], [43], [44, 290], [45], [46], [47], [48], [49, 241], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66, 88], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [83], [84], [85], [86], [87], [66, 88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99], [100], [101], [102], [103], [104], [105], [106], [107], [108], [109], [110], [111], [112], [113, 429], [114], [115], [116], [117], [118], [119], [120], [121, 195], [122], [123], [124], [125], [126], [127], [128], [129], [130], [131], [132], [133], [134], [40, 135], [136], [137], [138], [139], [140], [141], [142], [143], [144], [145], [146], [147], [148], [149], [150], [151], [15

## Covering number

### Wasserstein

In [60]:
from mip import Model, maximize, minimize, xsum

In [ ]:
m = Model()  # 数理モデル
x = []
# 変数
for i in range(N):
    s = "x"+str(i)
    x.append(m.add_var(s, lb=0, var_type="I"))

# 目的関数
z = x[0]
for i in range(1,N):
    z += x[i]

m.objective = minimize(z)

# 制約条件
for i in range(N):
    b = 0
    for j in nbh_w[i]:
        b += x[j]
    m += b >= 1
    
m.optimize()  # ソルバーの実行
print(m.objective.x)

# print("obj", m.objective.x, "x", x.x, "y", y.x)

90.0


### Regularized Wasserstein

In [ ]:
m = Model()  # 数理モデル
x = []
# 変数
for i in range(N):
    s = "x"+str(i)
    x.append(m.add_var(s, lb=0, var_type="I"))

# 目的関数
z = x[0]
for i in range(1,N):
    z += x[i]

m.objective = minimize(z)

# 制約条件
for i in range(N):
    b = 1
    for j in nbh_rw[i]:
        b += x[j]
    m += b >= 2
    
m.optimize()  # ソルバーの実行
print(m.objective.x)

489.0
